# Assignment

Data: 

Below represents sample coupon data for a retailer. Each column (tokenized value) is extracted from OfferDetails

In [1]:
import pandas as pd
import numpy as np

import re
from textblob import TextBlob

In [2]:
#Data
data = pd.read_csv('coupons_ner.csv', names = ['OfferDetails'])
print (data.shape)


(886, 1)


In [3]:
data.head()

,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...


## Exercise 1
* Write a function that takes OfferDetails as input, and returns FaceValue as output
* Run the function against all rows in the attached data and determine function accuracy (Higher the better)


Hint: For Exercise 1, regex function should do (ensure that all edge cases are taken care of)

In [4]:
def faceVal(sentence):
    '''
        Function find the $ value in the string and put return the total.
        Input: String 
        Output: Dollar total value
    '''
    sentence = sentence.lower()
    words = sentence.split(" ")
    
    saveMoneyWords = ['save',  'get', 'recieve', 'off']
    if 'save' not in words and 'get' not in words and 'recieve' not in words and 'off' not in words :
        return 'None'
    
    total_save = 0
    for index, word in enumerate(words):
        
        dd = re.findall(r"\$\d+(?:\.\d+)?", word)
       
        if '¢' in word:
            cent = re.findall(r"\d+(?:\.\d+)?\¢", word)
            total_cent = 0
            for k in cent:
                total_cent += float(re.findall('[0-9]+', k)[0])/100 
            dd.append('$'+str(total_cent))

        for k in dd: ## Check the previous and next word of dallor symbol
            if index == 0:
                if words[index+1] in saveMoneyWords:
                    total_save += float(re.findall(r"[-+]?\d*\.\d+|\d+", k)[0])
            elif index == len(words)-1:
                if words[index-1] in saveMoneyWords:
                    total_save += float(re.findall(r"[-+]?\d*\.\d+|\d+", k)[0])
            elif words[index+1] in saveMoneyWords or words[index-1] in saveMoneyWords:
                total_save += float(re.findall(r"[-+]?\d*\.\d+|\d+", k)[0])

    if total_save != 0:
        return '$'+str(total_save)
    else:
        return 'None'

In [5]:
data['FaceVal'] = data['OfferDetails'].apply(lambda x: faceVal(x))
print (data.head(20))

                                         OfferDetails FaceVal
0   Save $2.00 ONE Downy Liquid Fabric Conditioner...    $2.0
1   Save $2.00 ONE Tide PODS OR Tide Power PODS (e...    $2.0
2   Save $2.00 ONE Tide Laundry Detergent (exclude...    $2.0
3   SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...    $1.0
4   $3.00 OFF when you purchase any THREE (3) Pepp...    $3.0
5   SAVE $1.11 when you buy any ONE (1) Familly Si...   $1.11
6   SAVE $1.00 ON TWO when you buy TWO PACKAGES an...    $1.0
7   Save $1.00 on any TWO (2) Sargento® Natural Ch...    $1.0
8   $0.65 OFF On Any ONE (1) Oikos Greek Yogurt Cu...   $0.65
9   $2.00 OFF ONE (1) SMALL bag of Eight O'Clock® ...    $2.0
10  Save $0.50 ONE Tide Simply Laundry Detergent 3...    $0.5
11  SAVE $1.00 ON TWO when you buy TWO BOXES any f...    $1.0
12  $0.50 OFF on ONE (1) Frigo® Cheese Heads® 8ct ...    $0.5
13  Save $0.50 off ONE (1) jar of Pace® Salsa or P...    $0.5
14         $1.00 OFF on any THREE (3) noosa® yoghurts    $1.0
15     S

In [6]:
dataWithManualFaceVal = pd.read_csv('coupons_ner_fvalue.csv')
dataWithManualFaceVal.head()

,OfferDetails,FaceValue
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.0
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.0
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.0
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.0
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.0


In [7]:
dataWithManualFaceVal.fillna('None', inplace=True)
data.head()

,OfferDetails,FaceVal
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...,$2.0
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...,$2.0
2,Save $2.00 ONE Tide Laundry Detergent (exclude...,$2.0
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...,$1.0
4,$3.00 OFF when you purchase any THREE (3) Pepp...,$3.0


In [8]:
accuracy = [dataWithManualFaceVal['FaceValue'] == data['FaceVal']]

In [12]:
print (dataWithManualFaceVal['FaceValue'][100])
print (data['FaceVal'][100])


print (dataWithManualFaceVal['FaceValue'][884])
print (data['FaceVal'][884])
print ([dataWithManualFaceVal['FaceValue'][884] == data['FaceVal'][884]])



$2.0
$2.0
None
None
[True]


In [13]:
print ("Number of correctly predicting points:", sum(sum(accuracy)))
print ("Total number of data points:",len(data))
print ("Accuracy:", sum(sum(accuracy))/len(data)*100, '%')

Number of correctly predicting points: 859
Total number of data points: 886
Accuracy: 96.95259593679458 %


## Conclusion:
    1. It shows the 96.95% accuracy on face value prediction.